In [45]:
import pandas as pd
import numpy as np
import pickle
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import log_loss, roc_auc_score, auc, roc_curve
from sklearn.model_selection import train_test_split

DataPath = '../Data/data_origin'

dataset1 = pd.read_csv(f'{DataPath}/ProcessDataSet1.csv')
dataset1.label.replace(-1,0,inplace=True) 
dataset2 = pd.read_csv(f'{DataPath}/ProcessDataSet2.csv')
dataset2.label.replace(-1,0,inplace=True)
dataset3 = pd.read_csv(f'{DataPath}/ProcessDataSet3.csv')

dataset1.drop_duplicates(inplace=True)
dataset2.drop_duplicates(inplace=True)
dataset12 = pd.concat([dataset1,dataset2],axis=0)
dataset12_y = dataset12.label
dataset12_x = dataset12.drop(['user_id','label','day_gap_before','coupon_id','day_gap_after'],axis=1)      
                                         
dataset3.drop_duplicates(inplace=True)                       
dataset3_preds = dataset3[['user_id','coupon_id','date_received']]
dataset3_x = dataset3.drop(['user_id','coupon_id','date_received','day_gap_before','day_gap_after'],axis=1)

dataTrain = xgb.DMatrix(dataset12_x,label=dataset12_y)
dataTest = xgb.DMatrix(dataset3_x)

#性能评价函数
def myauc(test):
    testgroup = test.groupby(['coupon_id'])
    aucs = []
    for i in testgroup:
        tmpdf = i[1] 
        if len(tmpdf['label'].unique()) != 2:
            continue
        fpr, tpr, thresholds = roc_curve(tmpdf['label'], tmpdf['pred'], pos_label=1)
        aucs.append(auc(fpr,tpr))
    return np.average(aucs)


    

/anaconda3/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/anaconda3/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [16]:
dataset3

,user_id,coupon_id,discount_rate,distance,date_received,day_of_month,days_distance,discount_man,discount_jian,is_man_jian,...,day_gap_before,day_gap_after,is_weekend,weekday1,weekday2,weekday3,weekday4,weekday5,weekday6,weekday7
0,4129537,9983,0.833333,1.0,20160712,12,12,30.0,5.0,1,...,-1,-1,0,0,1,0,0,0,0,0
1,6949378,3429,0.833333,NaN,20160706,6,6,30.0,5.0,1,...,-1,-1,0,0,0,1,0,0,0,0
2,2166529,6928,0.900000,5.0,20160727,27,27,200.0,20.0,1,...,-1,-1,0,0,0,1,0,0,0,0
3,2166529,1808,0.900000,5.0,20160727,27,27,100.0,10.0,1,...,-1,-1,0,0,0,1,0,0,0,0
4,6172162,6500,0.966667,2.0,20160708,8,8,30.0,1.0,1,...,-1,-1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112798,5828093,10418,0.833333,10.0,20160716,16,16,30.0,5.0,1,...,-1,-1,1,0,0,0,0,0,1,0
112799,6626813,7595,0.966667,NaN,20160707,7,7,30.0,1.0,1,...,-1,-1,0,0,0,0,1,0,0,0
112800,6626813,7590,0.900000,NaN,20160712,12,12,50.0,5.0,1,...,-1,-1,0,0,1,0,0,0,0,0
112801,4547069,13602,0.833333,0.0,20160717,17,17,30.0,5.0,1,...,-1,-1,1,0,0,0,0,0,0,1


In [14]:
pd.read_csv(f"{DataPath}/ccf_offline_stage1_test_revised.csv",header=0,keep_default_na=False)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received
0,4129537,450,9983,30:5,1,20160712
1,6949378,1300,3429,30:5,null,20160706
2,2166529,7113,6928,200:20,5,20160727
3,2166529,7113,1808,100:10,5,20160727
4,6172162,7605,6500,30:1,2,20160708
...,...,...,...,...,...,...
113635,5828093,5717,10418,30:5,10,20160716
113636,6626813,1699,7595,30:1,null,20160707
113637,6626813,7321,7590,50:5,null,20160712
113638,4547069,760,13602,30:5,0,20160717


In [10]:
params={'booster':'gbtree',
	    'objective': 'rank:pairwise',
	    'eval_metric':'auc',
	    'gamma':0.1,
	    'min_child_weight':1.1,
	    'max_depth':5,
	    'lambda':10,
	    'subsample':0.7,
	    'colsample_bytree':0.7,
	    'colsample_bylevel':0.7,
	    'eta': 0.01,
	    'tree_method':'exact',
	    'seed':0,
	    'nthread':12
	    }
watchlist = [(dataTrain,'train')]
model = xgb.train(params,dataTrain,num_boost_round=3500,evals=watchlist)

model.save_model(f'{DataPath}/xgbmodel')
model=xgb.Booster(params)
model.load_model(f'{DataPath}/xgbmodel') 
#predict test set 
dataset3_preds1 = dataset3_preds
dataset3_preds1['label'] = model.predict(dataTest)
#标签归一化在[0，1]原作者代码这里有错
#修改前
#dataset3_preds.label = MinMaxScaler(copy=True,feature_range=(0,1)).fit_transform(dataset3_preds.label)
 
#修改后
dataset3_preds1.label = MinMaxScaler(copy=True,feature_range=(0,1)).fit_transform(dataset3_preds1.label.values.reshape(-1,1))
dataset3_preds1.sort_values(by=['coupon_id','label'],inplace=True)
dataset3_preds1.to_csv(f"{DataPath}/xgb_preds.csv",index=None,header=None)
print(dataset3_preds1.describe())

/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            user_id      coupon_id  date_received          label
count  1.128030e+05  112803.000000   1.128030e+05  112803.000000
mean   3.684618e+06    9064.658006   2.016072e+07       0.342602
std    2.126358e+06    4147.283515   9.017693e+00       0.133330
min    2.090000e+02       3.000000   2.016070e+07       0.000000
25%    1.843824e+06    5035.000000   2.016071e+07       0.259299
50%    3.683073e+06    9983.000000   2.016072e+07       0.321710
75%    5.525176e+06   13602.000000   2.016072e+07       0.409762
max    7.361024e+06   14045.000000   2.016073e+07       1.000000


In [2]:
model=xgb.Booster()
model.load_model(f'{DataPath}/xgbmodel') 

temp = dataset12[['coupon_id','label']].copy()
temp['pred'] =model.predict(xgb.DMatrix(dataset12_x))
temp.pred = MinMaxScaler(copy=True,feature_range=(0,1)).fit_transform(temp['pred'].values.reshape(-1,1))
print(myauc(temp))

0.7620884622821931


In [ ]:
params={'booster':'gbtree',
	    'objective': 'rank:pairwise',
	    'eval_metric':'auc',
	    'gamma':0.1,
	    'min_child_weight':1.1,
	    'max_depth':5,
	    'lambda':10,
	    'subsample':0.7,
	    'colsample_bytree':0.7,
	    'colsample_bylevel':0.7,
	    'eta': 0.01,
	    'tree_method':'exact',
	    'seed':0,
	    'nthread':12
	    }

cvresult = xgb.cv(params, dataTrain, num_boost_round=20000, nfold=5, metrics='auc', seed=0, callbacks=[
            xgb.callback.print_evaluation(show_stdv=False),
            xgb.callback.early_stop(50)
        ])
num_round_best = cvresult.shape[0] - 1
print('Best round num: ', num_round_best)

watchlist = [(dataTrain,'train')]
model1 = xgb.train(params,dataTrain,num_boost_round=num_round_best,evals=watchlist)

model1.save_model(f'{DataPath}/xgbmodel1')
print('------------------------train done------------------------------')

In [4]:
model1=xgb.Booster()
model1.load_model(f'{DataPath}/xgbmodel1') 

temp = dataset12[['coupon_id','label']].copy()
temp['pred'] =model1.predict(xgb.DMatrix(dataset12_x))
temp.pred = MinMaxScaler(copy=True,feature_range=(0,1)).fit_transform(temp['pred'].values.reshape(-1,1))
print(myauc(temp))


0.7981151470914131


In [ ]:
dataset3_preds2 = dataset3_preds
dataset3_preds2['label'] = model1.predict(dataTest)
dataset3_preds2.label = MinMaxScaler(copy=True,feature_range=(0,1)).fit_transform(dataset3_preds2.label.values.reshape(-1,1))
dataset3_preds2.sort_values(by=['coupon_id','label'],inplace=True)
dataset3_preds2.to_csv(f"{DataPath}/xgb_preds2.csv",index=None,header=None)
print(dataset3_preds2.describe())

In [ ]:
dataset3_preds3 = dataset3
dataset3_preds3['label'] = model1.predict(dataTest)
dataset3_preds3.label = MinMaxScaler(copy=True,feature_range=(0,1)).fit_transform(dataset3_preds3.label.values.reshape(-1,1))
dataset3_preds3.to_csv(f"{DataPath}/xgb_preds2_not_sorted.csv",index=None,header=True)
print(dataset3_preds3.describe())

In [82]:
def get_rate(row):
    if row.Is_man_jian == 0:
        return row.Discount_float
    else:
        return str(int(row.Discount_man)) + ":" + str(int(row.Discount_jian))

def correctify(df):
    result = df.copy()
    result.columns = [x.capitalize() for x in result.columns]
    result["Discount_float"] = result.Discount_rate
    result["Discount_rate_corrected"] = result.apply(get_rate, axis=1)
    result["Discount_rate"] = result["Discount_rate_corrected"]
    result.Distance.fillna("null", inplace=True)
    return result

In [83]:
dataset3_preds3_corr = correctify(dataset3_preds3)

In [7]:
feature_score = model1.get_fscore()
feature_score = sorted(feature_score.items(), key=lambda x:x[1],reverse=True)#value逆序排序

fs = []
for (key,value) in feature_score:
    fs.append("{0},{1}\n".format(key,value))
 
with open(f'{DataPath}/xgb_feature_score.csv','w') as f:
    f.writelines("feature,score\n")
    f.writelines(fs)

In [68]:
dataset3_preds3_corr[dataset3_preds3_corr.User_id == 5712798]

,User_id,Coupon_id,Discount_rate,Distance,Date_received,Day_of_month,Days_distance,Discount_man,Discount_jian,Is_man_jian,...,Weekday1,Weekday2,Weekday3,Weekday4,Weekday5,Weekday6,Weekday7,Label,Discount_float,Discount_rate_corrected
79286,5712798,3,0.98,1.0,20160709,9,9,50.0,1.0,1,...,0,0,0,0,0,1,0,0.148723,0.98,50:1


In [22]:
off_test = pd.read_csv(f"{DataPath}/ccf_offline_stage1_test_revised.csv",header=0,keep_default_na=False)

In [86]:
resultat = dataset3_preds3_corr.merge(off_test, how='left', on=['User_id', 'Coupon_id', 'Discount_rate', 'Distance', 'Date_received'])

In [92]:
resultat

,User_id,Coupon_id,Discount_rate,Distance,Date_received,Day_of_month,Days_distance,Discount_man,Discount_jian,Is_man_jian,...,Weekday2,Weekday3,Weekday4,Weekday5,Weekday6,Weekday7,Label,Discount_float,Discount_rate_corrected,Merchant_id
0,4129537,9983,30:5,1,20160712,12,12,30.0,5.0,1,...,1,0,0,0,0,0,0.228665,0.833333,30:5,NaN
1,6949378,3429,30:5,null,20160706,6,6,30.0,5.0,1,...,0,1,0,0,0,0,0.430711,0.833333,30:5,1300.0
2,2166529,6928,200:20,5,20160727,27,27,200.0,20.0,1,...,0,1,0,0,0,0,0.177080,0.900000,200:20,NaN
3,2166529,1808,100:10,5,20160727,27,27,100.0,10.0,1,...,0,1,0,0,0,0,0.240328,0.900000,100:10,NaN
4,6172162,6500,30:1,2,20160708,8,8,30.0,1.0,1,...,0,0,0,1,0,0,0.330845,0.966667,30:1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112869,5828093,10418,30:5,10,20160716,16,16,30.0,5.0,1,...,0,0,0,0,1,0,0.250761,0.833333,30:5,NaN
112870,6626813,7595,30:1,null,20160707,7,7,30.0,1.0,1,...,0,0,1,0,0,0,0.370142,0.966667,30:1,1699.0
112871,6626813,7590,50:5,null,20160712,12,12,50.0,5.0,1,...,1,0,0,0,0,0,0.350500,0.900000,50:5,7321.0
112872,4547069,13602,30:5,0,20160717,17,17,30.0,5.0,1,...,0,0,0,0,0,1,0.365258,0.833333,30:5,NaN


In [97]:
import os
from datetime import date, datetime
def exportResults(df, date):
    results_path_str = "results"
    filename = f"⁨results_{date.strftime('%Y-%m-%d_%H-%M-%S')}.csv"
    
    currentdir = os.getcwd()
    results_path = os.path.join(currentdir, results_path_str)
    full_path = os.path.join(results_path, filename)
    
    file = df[['User_id', 'Coupon_id', 'Date_received', 'Label']]
    file.to_csv(full_path, index = None, header=False)

In [ ]:
exportResults(resultat, datetime.now())